### Reinforcement learning tutorial using Python and Keras

####  The NChain example on Open AI Gym

In [1]:
import gym

In [29]:
env = gym.make('NChain-v0')
env.reset()

0

There are two possible actions in each state, move forward (action 0) and move backwards (action 1). When action 1 is taken, i.e. move backwards, there is an immediate reward of 2 given to the agent – and the agent is returned to state 0 (back to the beginning of the chain). However, when a move forward action is taken (action 0), there is no immediate reward until state 4. When the agent moves forward while in state 4, a reward of 10 is received by the agent. The agent stays in state 4 at this point also, so the reward can be repeated. There is also a random chance that the agent’s action is “flipped” by the environment (i.e. an action 0 is flipped to an action 1 and vice versa). 

In [30]:
env.step(1)

(0, 2, False, {})

In [32]:
for x in range(0,10):
    print(env.step(0))

(0, 2, False, {})
(1, 0, False, {})
(2, 0, False, {})
(3, 0, False, {})
(4, 0, False, {})
(4, 10, False, {})
(4, 10, False, {})
(4, 10, False, {})
(4, 10, False, {})
(4, 10, False, {})


The step() command returns 4 variables in a tuple, these are (in order):

The new state after the action
The reward due to the action
Whether the game is “done” or not – the NChain game is done after 1,000 steps
Debugging information – not relevant in this example

#### A first naive heuristic for reinforcement learning

In [ ]:
Table:
Rs0,a0 Rs0,a1
Rs1,a0 Rs1,a1
Rs2,a0 Rs2,a1
Rs3,a0 Rs3,a1
Rs4,a0 Rs4,a1


Each of the rows corresponds to the 5 available states in the NChain environment, and each column corresponds to the 2 available actions in each state – forward and backward, 0 and 1. The value in each of these table cells corresponds to some measure of reward that the agent has “learnt” occurs when they are in that state and perform that action. So, the value 
Rs0,a0  would be, say, the sum of the rewards that the agent has received when in the past they have been in state 0 and taken action 0. This table would then let the agent choose between actions based on the summated (or average, median etc. – take your pick) amount of reward the agent has received in the past when taking actions 0 or 1.


In [35]:
import numpy as np

In [36]:
def naive_sum_reward_agent(env, num_episodes=500):
    # this is the table that will hold our summated rewards for
    # each action in each state
    r_table = np.zeros((5, 2))
    for g in range(num_episodes):
        s = env.reset()
        done = False
        while not done:
            if np.sum(r_table[s, :]) == 0:
                # make a random selection of actions
                a = np.random.randint(0, 2)
            else:
                # select the action with highest cummulative reward
                a = np.argmax(r_table[s, :])
            new_s, r, done, _ = env.step(a)
            r_table[s, a] += r
            s = new_s
    return r_table

In [37]:
naive_sum_reward_agent(env, num_episodes=500)

array([[     0., 636810.],
       [     0., 127650.],
       [     0.,  25462.],
       [     0.,   5066.],
       [ 26022.,      0.]])

#### Delayed reward reinforcement learning


In [38]:
def q_learning_with_table(env, num_episodes=500):
    q_table = np.zeros((5, 2))
    y = 0.95
    lr = 0.8
    for i in range(num_episodes):
        s = env.reset()
        done = False
        while not done:
            if np.sum(q_table[s,:]) == 0:
                # make a random selection of actions
                a = np.random.randint(0, 2)
            else:
                # select the action with largest q value in state s
                a = np.argmax(q_table[s, :])
            new_s, r, done, _ = env.step(a)
            q_table[s, a] += r + lr*(y*np.max(q_table[new_s, :]) - q_table[s, a])
            s = new_s
    return q_table

In [39]:
q_learning_with_table(env, num_episodes=500)

array([[ 0.        , 22.48303275],
       [23.77290992,  0.        ],
       [25.01952495,  0.        ],
       [ 0.        , 24.63120007],
       [26.66844291,  0.        ]])

This output is strange, isn’t it? Again, we would expect at least the state 4 – action 0 combination to have the highest Q score, but it doesn’t.  We might also expect the reward from this action in this state to have cascaded down through the states 0 to 3. Something has clearly gone wrong – and the answer is that there isn’t enough exploration going on within the agent training method.

#### Q learning with ϵ-greedy action selection

In [40]:
def eps_greedy_q_learning_with_table(env, num_episodes=500):
    q_table = np.zeros((5, 2))
    y = 0.95
    eps = 0.5
    lr = 0.8
    decay_factor = 0.999
    for i in range(num_episodes):
        s = env.reset()
        eps *= decay_factor
        done = False
        while not done:
            # select the action with highest cummulative reward
            if np.random.random() < eps or np.sum(q_table[s, :]) == 0:
                a = np.random.randint(0, 2)
            else:
                a = np.argmax(q_table[s, :])
            # pdb.set_trace()
            new_s, r, done, _ = env.step(a)
            q_table[s, a] += r + lr * (y * np.max(q_table[new_s, :]) - q_table[s, a])
            s = new_s
    return q_table

In [41]:
eps_greedy_q_learning_with_table(env, num_episodes=500)

array([[39.18698907, 39.46869562],
       [40.96349376, 40.82985821],
       [46.97391879, 39.83781742],
       [43.45286595, 40.84857503],
       [52.51455787, 44.2583316 ]])

##### Comparing the methods

In [42]:
def test_methods(env, num_iterations=100):
    winner = np.zeros((3,))
    for g in range(num_iterations):
        m0_table = naive_sum_reward_agent(env, 500)
        m1_table = q_learning_with_table(env, 500)
        m2_table = eps_greedy_q_learning_with_table(env, 500)
        m0 = run_game(m0_table, env)
        m1 = run_game(m1_table, env)
        m2 = run_game(m2_table, env)
        w = np.argmax(np.array([m0, m1, m2]))
        winner[w] += 1
        print("Game {} of {}".format(g + 1, num_iterations))
    return winner

In [44]:
def run_game(table, env):
    s = env.reset()
    tot_reward = 0
    done = False
    while not done:
        a = np.argmax(table[s, :])
        s, r, done, _ = env.step(a)
        tot_reward += r
    return tot_reward

In [46]:
test_methods(env, num_iterations=3)

Game 1 of 3
Game 2 of 3
Game 3 of 3


array([0., 0., 3.])

#### Reinforcement learning with Keras


In [48]:
from keras.models import Sequential
from keras.layers import Dense, Activation, InputLayer

model = Sequential()
model.add(InputLayer(batch_input_shape=(1, 5)))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(2, activation='linear'))
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

Using TensorFlow backend.


In [49]:
num_episodes= 100

# now execute the q learning
y = 0.95
eps = 0.5
decay_factor = 0.999
r_avg_list = []
for i in range(num_episodes):
    s = env.reset()
    eps *= decay_factor
    if i % 100 == 0:
        print("Episode {} of {}".format(i + 1, num_episodes))
    done = False
    r_sum = 0
    while not done:
        if np.random.random() < eps:
            a = np.random.randint(0, 2)
        else:
            a = np.argmax(model.predict(np.identity(5)[s:s + 1]))
        new_s, r, done, _ = env.step(a)
        target = r + y * np.max(model.predict(np.identity(5)[new_s:new_s + 1]))
        target_vec = model.predict(np.identity(5)[s:s + 1])[0]
        target_vec[a] = target
        model.fit(np.identity(5)[s:s + 1], target_vec.reshape(-1, 2), epochs=1, verbose=0)
        s = new_s
        r_sum += r
    r_avg_list.append(r_sum / 1000)

Episode 1 of 100
